This script takes firing rate from cell_types_specimen_details.csv, precalculated volume from morphology_features.csv, nomenclature information from Allen_Classification_Paper_Sup_3.xlsx, as well as neuron_morphology calculation of cell volume based on SWC files. 

In [1]:
import numpy as np
import pandas as pd

In [14]:
pd.set_option('display.max_columns', 50)

In [19]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import os

In [4]:
# # If needs to interate through all swc files
# for subdir, dirs, files in os.walk('../data/cell_types/'):
#     for filename in files:
#         filepath = subdir + os.sep + filename

#         if filepath.endswith(".swc"):
#             print (filepath)

### Cell volume calculated with two methods

In [7]:
# import cell volume calculated with neuron_morphology on swc files of cells with full reconstruction
nm_df = pd.read_csv('../result/total_volume.tsv',sep='\t',index_col='Specimen_ID')

In [62]:
nm_df.rename(columns={'Total_Volume':'nm_volume'},inplace=True)

In [63]:
nm_df.shape

(367, 2)

In [39]:
vaa3d_df = pd.read_csv('../data/cell_types/morphology_features.csv',index_col='specimen_id')

In [56]:
# create a lite dataframe of cells with full reconstruction 
vaa3d_lite_df = vaa3d_df[vaa3d_df['superseded']==False].copy(deep=True) #32 cells have been superceded
vaa3d_lite_df = vaa3d_lite_df[vaa3d_lite_df['neuron_reconstruction_type']=='full']

In [65]:
vaa3d_lite_df.rename(columns={'total_volume':'vaa3d_volume'},inplace=True)

In [66]:
vaa3d_df.shape
vaa3d_lite_df.shape

(701, 29)

(367, 29)

In [67]:
volume_df = pd.merge(left=nm_df,right=vaa3d_lite_df,left_index=True,right_index=True,how='outer')

In [83]:
volume_df.shape

(367, 31)

### Import details and average firing rate

In [13]:
# import specimen_details, which also contains precalculated firing rate
# fr_df = pd.read_csv('../data/cell_types_specimen_details.csv')
# fr_df[fr_df.duplicated(subset=['specimen__id'],keep=False)] # No duplicated IDs
fr_df = pd.read_csv('../data/cell_types_specimen_details.csv',index_col='specimen__id')

In [16]:
# create a lite dataframe, keeping only cells that have average firing rate
fr_lite_df = fr_df.dropna(subset=['ef__avg_firing_rate'])

In [20]:
fr_df.shape
fr_lite_df.shape

(2333, 53)

(2176, 53)

### Each cell's me-type and nomenclature

In [70]:
# Import nomenclature 
nomen_df = pd.read_excel('../docs/Miller_et_al_2020_Sup_1/cell_set_nomenclature.xlsx',sheet_name='cell set nomenclature')

In [71]:
# Import classification of each cell
class_df = pd.read_excel('../docs/Allen_Classification_Paper_Sup_3.xlsx',sheet_name='Data')

In [72]:
# Create lite classification file, only keeping cells with a me-type
class_lite_df = class_df.dropna(subset=['me-type'])

In [73]:
class_df.shape
class_lite_df.shape

(1947, 8)

(452, 8)

In [74]:
# Matching nomenclature to cells with defined me-type (morphology-electrophysiology)
class_nomen_df = pd.merge(left=class_lite_df,right=nomen_df,left_on='me-type',right_on='cell_set_preferred_alias',how='left')

In [78]:
class_nomen_df.set_index('specimen_id',inplace=True)

In [79]:
class_nomen_df.shape

(452, 19)

In [80]:
class_nomen_df

,e-type,m-type,me-type,upright_angle,soma_distance_from_pia,estimated_shrinkage_factor,estimated_slice_angle,cell_set_preferred_alias,cell_set_ label,cell_set_ accession,cell_set_ aligned_alias,cell_set_ additional_alias,cell_set_ alias_assignee,cell_set_ alias_citation,cell_set_ structure,cell_set_ ontology_tag,taxonomy_id,species,modality
specimen_id,,,,,,,,,,,,,,,,,,,
313862022,Exc_3,Spiny_8,ME_Exc_6,150.319955,492.100275,2.977636,-25.453107,ME_Exc_6,ME 33,CS201906170_33,NaN,"Wide, short L6a & tufted (large basal) L5|RS a...",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
313862167,Inh_13,Aspiny_1,ME_Inh_15,119.629705,408.467273,2.030599,-21.213269,ME_Inh_15,ME 15,CS201906170_15,NaN,"L1-innervating ascending axon L2/3, L4|mid-wid...",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
313862373,Inh_2,Aspiny_4,ME_Inh_7,249.183146,926.652428,1.614714,-29.985693,ME_Inh_7,ME 07,CS201906170_7,NaN,"Wide axon, large dendrites L6|FS transient",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
314642645,Exc_3,Spiny_3,ME_Exc_17,233.914695,372.685943,2.251238,24.773125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314804042,Exc_3,Spiny_4,ME_Exc_16,238.336909,492.354331,2.874654,-12.595367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698217104,Exc_3,Spiny_10,ME_Exc_8,132.192873,528.334779,2.022481,14.721539,ME_Exc_8,ME 32,CS201906170_32,L5 NP,"Tufted (sparse basal) L5|RS adapting, large sag",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
698226504,Exc_3,Spiny_18,ME_Exc_2,141.786098,621.429913,2.515973,14.296986,ME_Exc_2,ME 35,CS201906170_35,NaN,Narrow L6a|RS adapting,Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
698230720,Exc_3,Spiny_10,ME_Exc_8,140.199770,577.460428,3.777499,12.829926,ME_Exc_8,ME 32,CS201906170_32,L5 NP,"Tufted (sparse basal) L5|RS adapting, large sag",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology


### Merging volume, firing rate, and nomenclature

In [81]:
# Cells with both volume and firing rate data avaliable
volume_fr_df = pd.merge(left=volume_df,right=fr_lite_df,left_index=True,right_index=True,how='inner')

In [86]:
volume_df.shape
fr_lite_df.shape
volume_fr_df.shape

(367, 31)

(2176, 53)

(346, 84)

In [87]:
# Assign avaliable nomenclature and/or me-type to cells, keeping cells without me-type
df = pd.merge(left=volume_fr_df,right=class_nomen_df,left_index=True,right_index=True,how='left')

In [88]:
df # Only mouse cells have me-type, but not human cells

,Species,nm_volume,average_bifurcation_angle_local,average_bifurcation_angle_remote,average_contraction,average_diameter,average_fragmentation,average_parent_daughter_ratio,hausdorff_dimension,id,max_branch_order,max_euclidean_distance,max_path_distance,neuron_reconstruction_type,number_bifurcations,number_branches,number_nodes,number_stems,number_tips,overall_depth,overall_height,overall_width,scale_factor_x,scale_factor_y,scale_factor_z,...,donor__disease_state,donor__race,donor__years_of_seizure_history,donor__species,donor__id,donor__name,e-type,m-type,me-type,upright_angle,soma_distance_from_pia,estimated_shrinkage_factor,estimated_slice_angle,cell_set_preferred_alias,cell_set_ label,cell_set_ accession,cell_set_ aligned_alias,cell_set_ additional_alias,cell_set_ alias_assignee,cell_set_ alias_citation,cell_set_ structure,cell_set_ ontology_tag,taxonomy_id,species,modality
313862167,Mouse,3074.544326,62.163118,NaN,0.930364,0.464887,59.486486,0.858529,NaN,759904803,5.0,247.737791,254.998821,full,19,43,2268,5,24,56.3931,383.260762,327.150383,0.1144,0.1144,0.28,...,NaN,NaN,NaN,Mus musculus,313401544,Sst-IRES-Cre;Ai14(IVSCC)-167638,Inh_13,Aspiny_1,ME_Inh_15,119.629705,408.467273,2.030599,-21.213269,ME_Inh_15,ME 15,CS201906170_15,NaN,"L1-innervating ascending axon L2/3, L4|mid-wid...",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
321708130,Mouse,4330.064060,84.566685,NaN,0.914024,0.317193,51.472222,0.914498,NaN,759896998,4.0,171.928465,204.964446,full,18,43,1970,7,25,76.8166,240.772522,276.926431,0.1144,0.1144,0.28,...,NaN,NaN,NaN,Mus musculus,320827555,Pvalb-IRES-Cre;Ai14(IVSCC)-172101,Inh_11,Aspiny_4,ME_Inh_6,165.768564,816.058572,3.401361,-10.165932,ME_Inh_6,ME 06,CS201906170_6,NaN,"Wide axon, large dendrites L6|FS transient & s...",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
323452196,Mouse,3878.987016,70.359932,NaN,0.931876,0.146125,40.250000,0.852783,NaN,513556739,3.0,197.455070,204.522930,full,6,18,581,6,12,47.0400,157.583433,271.542295,0.1144,0.1144,0.28,...,NaN,NaN,NaN,Mus musculus,320827615,Sst-IRES-Cre;Ai14(IVSCC)-172534,Inh_10,Aspiny_10,ME_Inh_11,132.392837,990.891895,2.417941,-20.936823,ME_Inh_11,ME 11,CS201906170_11,NaN,"Ascending axon L5, L6|FS delayed/pausing",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
323475862,Mouse,1675.121970,73.299960,NaN,0.896349,0.267869,44.678571,0.845183,NaN,760196580,6.0,198.757384,224.942535,full,14,34,1433,6,20,68.1800,247.547693,291.582190,0.1144,0.1144,0.28,...,NaN,NaN,NaN,Mus musculus,320827615,Sst-IRES-Cre;Ai14(IVSCC)-172534,Inh_5,Aspiny_9,ME_Inh_26,33.092525,607.239560,3.670988,-4.124467,ME_Inh_26,ME 26,CS201906170_26,NaN,"Ascending small axon L5, L4|mid-width AP trans...",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
324266189,Mouse,3369.223223,73.406835,NaN,0.885431,0.471252,55.750000,0.849701,NaN,657301037,5.0,183.250754,217.333733,full,12,28,1402,4,16,94.2998,331.170180,263.451962,0.1144,0.1144,0.28,...,NaN,NaN,NaN,Mus musculus,322489513,Sst-IRES-Cre;Ai14(IVSCC)-173191,Inh_6,Aspiny_1,ME_Inh_25,154.796755,321.145937,2.211975,-2.207497,ME_Inh_25,ME 25,CS201906170_25,NaN,"L1-innervating ascending axon L2/3, L4 & ascen...",Nathan Gouwens,10.1038/s41593-019-0417-0,primary visual cortex,UBERON_0002436,CCN201906170,Mouse,Morphology|Electrophysiology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623185845,Mouse,6004.160584,69.386618,NaN,0.850527,0.672994,47.195122,0.990384,NaN,656081412,7.0,296.215491,337.868738,full,41,88,3962,6,47,147.2033,400.651267,412.550768,0.1144,0.1144,0.28,...,NaN,NaN,NaN,Mus mu

In [89]:
df.to_csv('../result/volume_FR_nomenclature.tsv', sep='\t')

In [90]:
set(df['Species'])

{'Human', 'Mouse'}

In [91]:
human_df=df[df['Species']=='Human']

In [93]:
human_df.to_csv('../result/human_volume_FR_nomenclature.tsv', sep='\t')

In [92]:
mouse_df=df[df['Species']=='Mouse']

In [94]:
mouse_df.to_csv('../result/mouse_volume_FR_nomenclature.tsv', sep='\t')